In [ ]:
# %pip install --upgrade  domolibrary

In [ ]:
import domolibrary

domolibrary.__version__

'0.3.46'

# 0. Get Sample Data and generate auth object


In [ ]:
import csv
import pandas as pd


with open("dataset.csv", "r", encoding="utf-8-sig") as file:
    csv_reader = csv.DictReader(file)
    data = [row for row in csv_reader]

for obj in data:
    obj.update(
        {
            "email": obj["email"].replace(" ", ""),
            "manager_email": obj["manager_email"].replace(" ", ""),
        },
    )  # clean sample dataset so email is a valid email

pd.DataFrame(data)[0:5]

,email,Role,FranchiseID,StoreID,EmpID,manager_email
0,CEO1@fz.com,CEO,0,0,1,jae@onyxreporting.com
1,Owner2@fz.com,Owner,1,2,2,CEO1@fz.com
2,RestaurantManager3@fz.com,Restaurant Manager,1,24,3,Owner2@fz.com
3,RestaurantManager4@fz.com,Restaurant Manager,1,7,4,Owner2@fz.com
4,RestaurantManager5@fz.com,Restaurant Manager,1,11,5,Owner2@fz.com


In [ ]:
import os
import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# 1. Create Custom Attributes in Domo Instance Config

User attributes, will be custom properties that we can manage and use to create dynamic pdp policies and groups

- First we parse the CSV to identify new properties to create


In [ ]:
properties_to_create = list(
    set(
        [
            key
            for obj in data
            for key in obj.keys()
            if key.lower() not in ["email", "manager_email", "empid", "role"]
        ]
    )
)

properties_to_create

['FranchiseID', 'StoreID']

In [ ]:
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoInstanceConfig as dmic
import domolibrary.classes.DomoInstanceConfig_UserAttribute as dmua
import domolibrary.utils.chunk_execution as ce
from typing import List


async def upsert_property(domo_config, property_name: str):
    """will create a custom property in Domo"""

    return await domo_config.user_attributes.upsert(
        attribute_id=f"fz{property_name}",
        name=property_name,
        description="FZ_POC",
        debug_api=False,
        debug_prn=False,
    )


async def upsert_properties(
    property_ls: List[str], auth: dmda.DomoAuth  # list of property names to create
):
    """will create a list of custom properties from a provided list of property names"""

    domo_config = dmic.DomoInstanceConfig(auth=auth)

    return await ce.gather_with_concurrency(
        *[upsert_property(domo_config, property_name) for property_name in property_ls],
        n=10,
    )


await upsert_properties(properties_to_create, auth=token_auth)

[UserAttribute(id='fzFranchiseID', name='FranchiseID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='fzStoreID', name='StoreID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True)]

## validate that custom properties have been created

- The `DomoInstanceConfig` class has a property `user_attributes` that can be used to interact with all user attributes (including standard Domo Attributes, IDP created attributes, and custom attributes)


In [ ]:
async def get_custom_properties(
    auth: dmda.DomoAuth, prop_prefix: str, return_all: bool = False
):
    domo_config = dmic.DomoInstanceConfig(auth=auth)
    props = await domo_config.user_attributes.get_attributes()

    if return_all:
        return props
    return [prop for prop in props if prop.id.startswith(prop_prefix)]


await get_custom_properties(auth=token_auth, prop_prefix="fz", return_all=False)

[UserAttribute(id='fzManagerEmail', name='ManagerEmail', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='fzStoreID', name='StoreID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='fzFranchiseID', name='FranchiseID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True)]

# 2. Extend User Properties

- the UserProperty_Type enum from domolibrary has a list of standard user properties.
- extend the `UserProperty_Type` enum with the newly created user attributes

NOTE: UserProperty_Type and the idea of a custom attribute are duplicate and in a later implementation of DomoLibrary might be rolled into one class.

For the time being, first you create the custom property, then you use UserProperty to update the DomoUser class


In [ ]:
# %pip install aenum

In [ ]:
from domolibrary.classes.DomoUser import UserProperty, UserProperty_Type
from pprint import pprint

# predefined properties we can update
# if a custom property hasn't been added we'll extended it with the aenum library

pprint(
    {
        "pedefined_properties": [
            {member.name: member.value} for member in UserProperty_Type
        ]
    }
)

{'pedefined_properties': [{'display_name': 'displayName'},
                          {'email_address': 'emailAddress'},
                          {'phone_number': 'phoneNumber'},
                          {'title': 'title'},
                          {'department': 'department'},
                          {'web_landing_page': 'webLandingPage'},
                          {'web_mobile_landing_page': 'webMobileLandingPage'},
                          {'role_id': 'roleId'},
                          {'employee_id': 'employeeId'},
                          {'employee_number': 'employeeNumber'},
                          {'hire_date': 'hireDate'},
                          {'reports_to': 'reportsTo'},
                          {'fzFranchiseID': 'fzFranchiseID'},
                          {'fzStoreID': 'fzStoreID'}]}


In [ ]:
from aenum import extend_enum  # use aenum to extend an existing enum

if "fzFranchiseID" not in UserProperty_Type.__members__:
    extend_enum(UserProperty_Type, "fzFranchiseID", "fzFranchiseID")

if "fzStoreID" not in UserProperty_Type.__members__:
    extend_enum(UserProperty_Type, "fzStoreID", "fzStoreID")

# validate properties have been extended
pprint(
    {
        "pedefined_properties": [
            {member.name: member.value} for member in UserProperty_Type
        ]
    }
)

{'pedefined_properties': [{'display_name': 'displayName'},
                          {'email_address': 'emailAddress'},
                          {'phone_number': 'phoneNumber'},
                          {'title': 'title'},
                          {'department': 'department'},
                          {'web_landing_page': 'webLandingPage'},
                          {'web_mobile_landing_page': 'webMobileLandingPage'},
                          {'role_id': 'roleId'},
                          {'employee_id': 'employeeId'},
                          {'employee_number': 'employeeNumber'},
                          {'hire_date': 'hireDate'},
                          {'reports_to': 'reportsTo'},
                          {'fzFranchiseID': 'fzFranchiseID'},
                          {'fzStoreID': 'fzStoreID'}]}


At this point the custom properties should have been added to the UserProperty_Type enum and can be used to upgrade domousers


In [ ]:
test_user = data[0]
test_user

{'email': 'CEO1@fz.com',
 'Role': 'CEO',
 'FranchiseID': '0',
 'StoreID': '0',
 'EmpID': '1',
 'manager_email': 'jae@onyxreporting.com'}

In [ ]:
import domolibrary.classes.DomoUser as dmdu


async def create_user_property(
    franchise_id, store_id, employee_id, manager_email, auth: dmda.DomoAuth
):
    """creates a property that can be passed to DomoUser class to update a property in Domo"""

    manager = await dmdu.DomoUsers.by_email(
        email_ls=[manager_email], only_allow_one=False, auth=auth
    )

    manager = manager[0]
    print(manager.email_address)
    assert manager

    return [
        UserProperty(UserProperty_Type.fzFranchiseID, franchise_id),
        UserProperty(UserProperty_Type.fzStoreID, store_id),
        UserProperty(UserProperty_Type.employee_number, employee_id),
        UserProperty(UserProperty_Type.reports_to, manager.id),
    ]


test_property = await create_user_property(
    franchise_id=test_user["FranchiseID"],
    store_id=test_user["StoreID"],
    employee_id=test_user["EmpID"],
    manager_email=test_user["manager_email"],
    auth=token_auth,
)

[prop.to_json() for prop in test_property]

jae@onyxreporting.com


[{'key': 'fzFranchiseID', 'values': ['0']},
 {'key': 'fzStoreID', 'values': ['0']},
 {'key': 'employeeNumber', 'values': ['1']},
 {'key': 'reportsTo', 'values': ['1893952720']}]

# 3. create / upsert users with custom properties

- first we will create the user then update the properties


In [ ]:
import domolibrary.classes.DomoUser as dmdu
import asyncio


async def upsert_user(
    email,
    role_id,
    franchise_id,
    store_id,
    employee_id,
    manager_email,
    auth: dmda.DomoAuth,
    display_name=None,
):
    """upsert a user by email, then updates properties franchise_id and role_id"""

    domo_user = await dmdu.DomoUsers.upsert_user(
        email_address=email,
        display_name=display_name or f"fz_{email}",
        role_id=role_id,
        auth=auth,
        debug_api=False,
    )

    property_ls = await create_user_property(
        franchise_id=franchise_id,
        store_id=store_id,
        employee_id=employee_id,
        manager_email=manager_email,
        auth=auth,
    )
    try:
        await asyncio.sleep(4)
        await domo_user.update_properties(property_ls)

    except Exception as e:
        print(e)

    return domo_user


print(test_user)
print("\n")
domo_user = await upsert_user(
    email=test_user["email"],
    role_id=5,
    franchise_id=test_user["FranchiseID"],
    store_id=test_user["StoreID"],
    employee_id=test_user["EmpID"],
    manager_email=test_user["manager_email"],
    auth=token_auth,
)

## this view of the user will not reflect the updated attribtues, as the get_by_id() API does not retrieve custom attributes
# instead you have to use the Query API to search the datacenter for full details

pprint(domo_user)

NameError: name 'dmda' is not defined

In [ ]:
await ce.gather_with_concurrency(
    *[
        upsert_user(
            email=user_obj["email"],
            role_id=5,
            franchise_id=user_obj["FranchiseID"],
            store_id=user_obj["StoreID"],
            employee_id=user_obj["EmpID"],
            manager_email=user_obj["manager_email"],
            auth=token_auth,
        )
        for user_obj in data
    ],
    n=10
)

Owner2@fz.com
Owner2@fz.com
CEO1@fz.com
Owner7@fz.com
Owner7@fz.com
jae@onyxreporting.com
CEO1@fz.com
Owner2@fz.com
Owner2@fz.com
Owner7@fz.com
Owner7@fz.com
CEO1@fz.com
Owner12@fz.com
Owner12@fz.com
Owner12@fz.com
Owner12@fz.com
Owner12@fz.com


User_CrudError: 🛑  User_CrudError 🛑 - function: update_user || 1009228622 || status 400 || Bad Request at domo-community

In [ ]:
for user_obj in data:
    print(user_obj)
    await upsert_user(
        email=user_obj["email"],
        role_id=5,
        franchise_id=user_obj["FranchiseID"],
        store_id=user_obj["StoreID"],
        employee_id=user_obj["EmpID"],
        manager_email=user_obj["manager_email"],
        auth=token_auth,
    )

{'email': 'CEO1@fz.com', 'Role': 'CEO', 'FranchiseID': '0', 'StoreID': '0', 'EmpID': '1', 'manager_email': 'jae@onyxreporting.com'}
jae@onyxreporting.com
{'email': 'Owner2@fz.com', 'Role': 'Owner', 'FranchiseID': '1', 'StoreID': '2', 'EmpID': '2', 'manager_email': 'CEO1@fz.com'}
CEO1@fz.com
{'email': 'RestaurantManager3@fz.com', 'Role': 'Restaurant Manager', 'FranchiseID': '1', 'StoreID': '24', 'EmpID': '3', 'manager_email': 'Owner2@fz.com'}
Owner2@fz.com
{'email': 'RestaurantManager4@fz.com', 'Role': 'Restaurant Manager', 'FranchiseID': '1', 'StoreID': '7', 'EmpID': '4', 'manager_email': 'Owner2@fz.com'}
Owner2@fz.com
{'email': 'RestaurantManager5@fz.com', 'Role': 'Restaurant Manager', 'FranchiseID': '1', 'StoreID': '11', 'EmpID': '5', 'manager_email': 'Owner2@fz.com'}
Owner2@fz.com
{'email': 'RestaurantManager6@fz.com', 'Role': 'Restaurant Manager', 'FranchiseID': '1', 'StoreID': '8', 'EmpID': '6', 'manager_email': 'Owner2@fz.com'}
Owner2@fz.com
{'email': 'Owner7@fz.com', 'Role': 'Ow